# Groundwater Forcing Generator

This notebook demonstrates how to generate groundwater forcing files for FVCOM using the `GroundwaterNetCDFGenerator`.

In [ ]:
import numpy as np
from pathlib import Path
from xfvcom.io import GroundwaterNetCDFGenerator

## 1. Constant Values Example

Generate a groundwater forcing file with constant values for all nodes and times:

In [ ]:
# Assuming you have a grid file
grid_file = Path("../tests/data/sample_grid.nc")  # Update with your grid file

# Create generator with constant values
gen = GroundwaterNetCDFGenerator(
    grid_nc=grid_file,
    start="2025-01-01T00:00:00Z",
    end="2025-01-07T00:00:00Z",
    dt_seconds=3600,  # hourly
    flux=0.001,       # m³/s
    temperature=15.0, # °C
    salinity=0.5,     # PSU
)

# Write to file
output_file = Path("groundwater_constant.nc")
gen.write(output_file)
print(f"Generated: {output_file}")

## 2. Node-Varying Values Example

Create forcing with different values for each node (but constant in time):

In [ ]:
# First, check how many nodes in the grid
import xarray as xr
with xr.open_dataset(grid_file) as ds:
    n_nodes = ds.dims["node"]
    print(f"Grid has {n_nodes} nodes")

# Create node-specific values
flux_by_node = np.random.uniform(0.0, 0.01, n_nodes)  # Random flux values
temp_by_node = np.random.uniform(10.0, 20.0, n_nodes) # Random temperatures

# Create generator
gen = GroundwaterNetCDFGenerator(
    grid_nc=grid_file,
    start="2025-01-01T00:00:00Z",
    end="2025-01-02T00:00:00Z",
    dt_seconds=3600,
    flux=flux_by_node,
    temperature=temp_by_node,
    salinity=0.0,  # Fresh water
)

output_file = Path("groundwater_nodes.nc")
gen.write(output_file)
print(f"Generated: {output_file}")

## 3. Time-Varying Values Example

Create forcing that varies in both space and time:

In [ ]:
# Define time parameters
start = "2025-01-01T00:00:00Z"
end = "2025-01-03T00:00:00Z"
dt_hours = 6  # 6-hourly data

# Calculate number of time steps
import pandas as pd
times = pd.date_range(start, end, freq=f"{dt_hours}h", inclusive="both")
n_times = len(times)
print(f"Time steps: {n_times}")

# Create time-varying flux (e.g., tidal influence)
flux_data = np.zeros((n_nodes, n_times))
for i in range(n_nodes):
    # Sinusoidal variation with different phase for each node
    phase = i * 2 * np.pi / n_nodes
    flux_data[i, :] = 0.005 * (1 + np.sin(2 * np.pi * np.arange(n_times) / 4 + phase))

# Create generator
gen = GroundwaterNetCDFGenerator(
    grid_nc=grid_file,
    start=start,
    end=end,
    dt_seconds=dt_hours * 3600,
    flux=flux_data,
    temperature=15.0,  # Constant temperature
    salinity=0.0,      # Fresh water
)

output_file = Path("groundwater_timevarying.nc")
gen.write(output_file)
print(f"Generated: {output_file}")

## 4. Using the CLI

You can also use the command-line interface:

In [ ]:
# Display CLI help
!xfvcom-make-groundwater-nc --help

### CLI Examples:

```bash
# Constant values
xfvcom-make-groundwater-nc grid.nc \
    --start 2025-01-01T00:00Z --end 2025-01-07T00:00Z \
    --flux 0.001 --temperature 15.0 --salinity 0.0

# Using CSV files for node-specific values
xfvcom-make-groundwater-nc grid.nc \
    --start 2025-01-01 --end 2025-12-31 \
    --start-tz Asia/Tokyo \
    --flux groundwater_flux_by_node.csv \
    --temperature 10.0 --salinity 0.0

# For .dat grid files, specify UTM zone
xfvcom-make-groundwater-nc grid.dat --utm-zone 54 \
    --start 2025-01-01T00:00Z --end 2025-01-02T00:00Z \
    --flux 0.005 --temperature 18.0 --salinity 32.0
```

## 5. Verify Output

Check the generated NetCDF file:

In [ ]:
# Read with decode_times=False to handle FVCOM time format
with xr.open_dataset("groundwater_constant.nc", decode_times=False) as ds:
    print(ds)
    print("\nVariables:")
    for var in ds.data_vars:
        print(f"  {var}: {ds[var].dims} - {ds[var].attrs.get('long_name', '')}")